In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk import word_tokenize

In [2]:
database=pd.read_csv("train.csv")
document=database.text.values
target=database.airline_sentiment.values
database

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [3]:
tokenized_words=[word_tokenize(w) for w in document]

In [4]:
no_stop=[[words for words in w if words not in stopwords.words("english")] for w in tokenized_words]

In [5]:
no_pun=[[words for words in w if words not in string.punctuation] for w in no_stop]
from nltk.tokenize.treebank import TreebankWordDetokenizer
clean_doc=[]
for i in range(len(no_pun)):
    words=TreebankWordDetokenizer().detokenize(no_pun[i])
    clean_doc.append(words)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(max_features=10,ngram_range=(1,2))
train_data=vectorizer.fit_transform(clean_doc)
train_data.todense()

matrix([[0, 0, 1, ..., 1, 0, 0],
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 1, ..., 0, 1, 0]], dtype=int64)

In [7]:
y_train=[]
for i in range(len(target)):
    if target[i]=="negative":
        y_train.append(0)
    if target[i]=="neutral":
        y_train.append(1)
    if target[i]=="positive":
        y_train.append(2)

In [12]:
from sklearn.naive_bayes import GaussianNB
clf=SVC()
clf.fit(train_data,y_train)
x_test_pd=pd.read_csv("test.csv")

In [9]:
test_document=x_test_pd.text.values
tokenized_words_t=[word_tokenize(w) for w in test_document]
no_stop_t=[[words for words in w if words not in stopwords.words("english")] for w in tokenized_words_t]
no_pun_t=[[words for words in w if words not in string.punctuation] for w in no_stop_t]
from nltk.tokenize.treebank import TreebankWordDetokenizer
clean_doc_t=[]
for i in range(len(no_pun_t)):
    words=TreebankWordDetokenizer().detokenize(no_pun_t[i])
    clean_doc_t.append(words)

In [10]:
vectorizer_t=CountVectorizer(max_features=10,ngram_range=(1,2))
test_data=vectorizer.fit_transform(clean_doc_t)

In [24]:
y_predict=clf.predict(test_data)
clf.score(train_data,y_train)
y_predict_text=[]
for i in range(len(y_predict)):
    if y_predict[i]==0:
        y_predict_text.append("negative")
    if y_predict[i]==1:
        y_predict_text.append("neutral")
    if y_predict[i]==2:
        y_predict_text.append("positive")
y_predict_np=np.array(y_predict_text)
y_predict_np

array(['negative', 'negative', 'negative', ..., 'neutral', 'negative',
       'negative'], dtype='<U8')

In [28]:
np.savetxt("submit.csv",y_predict_np,fmt="%s")
